# Old ways of using OpenAI API

## via openai python ChatCompletion class

In [47]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

completion=openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"system","content":"You are a helpful assistant."},
        {"role":'user',"content":"Hello"}
               
               ]
)

print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "Hi there! How can I assist you today?"
}


## Via CURL command

```bash
curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-3.5-turbo",
    "messages": [
      {"role": "system", "content": "You are a helpful assistant."}, 
      {"role": "user", "content": "Hello!"}
      ]
  }'
```

## Via request module

```python
OPENAI_API_KEY=openai.api_key = os.getenv("OPENAI_API_KEY")


import requests

headers  ={

    "Contenet-Type" : "application/json",
    "Authorization" : "Bearer " + OPENAI_API_KEY
}
json_data = {
    'model':'gpt-3.5-turbo',
    'messages': [
        {'role': 'system', 'content': 'You are ahelpful assistant.' },
        {'role': 'user', 'content': 'Hello!' },
    ]
}
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=json_data
 )

```


# Weather Chatbot


## Checkout the following example - 

```bash
curl https://api.openai.com/v1/chat/completions -u :$OPENAI_API_KEY -H 'Content-Type: application/json' -d '{
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {"role": "user", "content": "What is the weather like in Boston?"}
  ],
  "functions": [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
}'
```

focus on the function keyword and the details under it can be found in a functions docstring. So from the above function description,
you can assume the `get_current_weather` as follows - 

```python
def get_current_weather(location, unit):
    """Get the current weather in a given location

    Args:
        location (str): The city and state, e.g. San Francisco, CA
        unit (str): ["celsius", "fahrenheit"]
    """
    pass
```

In [48]:
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GPT_MODEL = 'gpt-3.5-turbo-0613' #new model for function calling abilities



In [49]:
import requests

def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        'Content_Type': 'application/Json',
        'Authorization': 'Bearer ' + OPENAI_API_KEY,
    }
    json_data ={
        "model":model,
        "messages":messages
    }
    if functions is not None:
        json_data.update({"functions":functions})
    if function_call is not None:
        json_data.update({"fuction_call":function_call})

    try:
        response=requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers = headers,
            json = json_data
        )
        return response
    except Exception as e:
        print(f"\n>>>Unable to generate ChatCompletion response")
        print(f"\n>>> Exception: {e}")
        raise e




In [50]:
messages = [
    {'role':'system','content':"you are a helpful assistant"},
    {'role':'user','content':'Hi, there'},
]

chat_response = chat_completion_request(messages=messages)

print(chat_response)
chat_response.json()


<Response [200]>


{'id': 'chatcmpl-7YSQZw6gy4d0PsfqcpqBduIXPROUM',
 'object': 'chat.completion',
 'created': 1688446783,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello! How can I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 19, 'completion_tokens': 9, 'total_tokens': 28}}

In [51]:
assistant_message = chat_response.json()
assistant_message['choices'][0]['message']['content']

'Hello! How can I assist you today?'

## Prompting


In [52]:
creator_name = "mach-42"

system_instruction = f"""
You are Weather bot created by {creator_name}, \
You first greet the customer, and you MUST introduce yourself as -
"Hi I am Weather bot created by {creator_name}! How may I assist you today?" \
an automated service to tell Weather infomation \
based on the location provided by the user. \
then collects the location information\
Don't make assumptions about what \
values to plug into functions. \
Ask for clarification if a user request is ambiguous.
You respond in a short, very conversational friendly style. \
"""

In [53]:
messages = [{'role':'system',"content":system_instruction}]
messages

[{'role': 'system',
  'content': '\nYou are Weather bot created by mach-42, You first greet the customer, and you MUST introduce yourself as -\n"Hi I am Weather bot created by mach-42! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon\'t make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nYou respond in a short, very conversational friendly style. '}]

In [54]:
functions = [
    {
        "name":"get_current_weather", # Name of the function
        "description":"This weather function will fetch the current weather report",
        'parameters':{
            "type":"object",
            'properties':{ # 1st arguement
                'location':{ #arguement name
                    'type':'string', #arguement type
                    'description':'The city and state, eg- new Delgi, Bengluru' # description of a function with example
                }
            },
            'required':['location'] # required requirement i.e ist arg is required for this function
        }


    }
]

In [55]:
def get_current_weather(location):
    RAPID_API_KEY = os.getenv('RAPID_API_KEY')
    try:

        url = "https://weatherapi-com.p.rapidapi.com/current.json"

        querystring = {"q":location}
        print(f"\n>>>> got the querystring as: {querystring}")


        headers = {
            "X-RapidAPI-Key": RAPID_API_KEY,
            "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        response_json = response.json()
        print(f"\n>>>> got the RAPID API response as: \n{response_json}")
        return response_json

    except Exception as e:
        print("Exception occured in fetching rapid api as : {e}")
        raise e
    

In [56]:
user_message = "Hi there!"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)
chat_response.json()

{'id': 'chatcmpl-7YSQaAU9UVwgcvhFXowZxocILDmYI',
 'object': 'chat.completion',
 'created': 1688446784,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hi! How can I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 170, 'completion_tokens': 10, 'total_tokens': 180}}

### Messge Extraction

In [64]:
messages=[] #Empty the messages list to avoid previous stored data
messages.append({'role':'system',"content":system_instruction})
user_message = "Hi, there."
messages.append({'role': 'user', 'content': user_message})

functions = [
    {
        "name":"get_current_weather", # Name of the function
        "description":"This weather function will fetch the current weather report",
        'parameters':{
            "type":"object",
            'properties':{ # 1st arguement
                'location':{ #arguement name
                    'type':'string', #arguement type
                    'description':'The city and state, eg- new Delgi, Bengluru' # description of a function with example
                }
            },
            'required':['location'] # required requirement i.e ist arg is required for this function
        }
    }
]

# Calling the chatgpt api response by passing the message and function
chat_reasponse = chat_completion_request(messages,functions)

# Extracting the message
assistant_message = chat_reasponse.json()['choices'][0]['message']
print(f"\nAssistant Message: \n{assistant_message}") 



Assistant Message: 
{'role': 'assistant', 'content': 'Hi there! How can I assist you today? I am Weather bot created by mach-42!'}


In [65]:
messages.append(assistant_message)
messages

[{'role': 'system',
  'content': '\nYou are Weather bot created by mach-42, You first greet the customer, and you MUST introduce yourself as -\n"Hi I am Weather bot created by mach-42! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon\'t make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nYou respond in a short, very conversational friendly style. '},
 {'role': 'user', 'content': 'Hi, there.'},
 {'role': 'assistant',
  'content': 'Hi there! How can I assist you today? I am Weather bot created by mach-42!'}]

In [74]:
user_message = "I want to know the weather at Bengaluru."
messages.append({'role':'user','content':user_message})
chat_reasponse = chat_completion_request(messages,functions)


print(f"\nComplete Chat Response: \n{chat_reasponse.json()}")
assistant_message=chat_reasponse.json()['choices'][0]['message']
print(f"\nAssistant Message: \n{assistant_message}")




Complete Chat Response: 
{'id': 'chatcmpl-7YT0vUbJoWPBlbBMx0QdwFBfvwDxD', 'object': 'chat.completion', 'created': 1688449037, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Bengaluru"\n}'}}, 'finish_reason': 'function_call'}], 'usage': {'prompt_tokens': 225, 'completion_tokens': 19, 'total_tokens': 244}}

Assistant Message: 
{'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Bengaluru"\n}'}}


In [75]:
chat_reasponse.json()

{'id': 'chatcmpl-7YT0vUbJoWPBlbBMx0QdwFBfvwDxD',
 'object': 'chat.completion',
 'created': 1688449037,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'get_current_weather',
     'arguments': '{\n  "location": "Bengaluru"\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 225, 'completion_tokens': 19, 'total_tokens': 244}}

In [76]:
messages

[{'role': 'system',
  'content': '\nYou are Weather bot created by mach-42, You first greet the customer, and you MUST introduce yourself as -\n"Hi I am Weather bot created by mach-42! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon\'t make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nYou respond in a short, very conversational friendly style. '},
 {'role': 'user', 'content': 'Hi, there.'},
 {'role': 'assistant',
  'content': 'Hi there! How can I assist you today? I am Weather bot created by mach-42!'},
 {'role': 'user', 'content': 'I want to know the weather at Prayagraj.'},
 {'role': 'user', 'content': 'I want to know the weather at Bengaluru.'}]

In [77]:
assistant_message = chat_reasponse.json()['choices'][0]['message']
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "Bengaluru"\n}'}}

In [78]:
import json

def execute_function_call(assistant_message):
    if assistant_message.get('function_call').get('name') == "get_current_weather":
        location = json.loads(assistant_message.get('function_call').get('arguments'))['location']
        results = get_current_weather(location)
    else:
        results = f"Error: function {assistant_message['function_call']['name']} dosent exist."
    return results



In [82]:
results=execute_function_call(assistant_message)
content = json.dumps(results)
content


>>>> got the querystring as: {'q': 'Bengaluru'}

>>>> got the RAPID API response as: 
{'location': {'name': 'Bengaluru', 'region': 'Karnataka', 'country': 'India', 'lat': 12.98, 'lon': 77.58, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1688449268, 'localtime': '2023-07-04 11:11'}, 'current': {'last_updated_epoch': 1688448600, 'last_updated': '2023-07-04 11:00', 'temp_c': 24.0, 'temp_f': 75.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 12.5, 'wind_kph': 20.2, 'wind_degree': 240, 'wind_dir': 'WSW', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.4, 'precip_in': 0.02, 'humidity': 78, 'cloud': 50, 'feelslike_c': 25.9, 'feelslike_f': 78.5, 'vis_km': 6.0, 'vis_miles': 3.0, 'uv': 5.0, 'gust_mph': 23.5, 'gust_kph': 37.8}}


'{"location": {"name": "Bengaluru", "region": "Karnataka", "country": "India", "lat": 12.98, "lon": 77.58, "tz_id": "Asia/Kolkata", "localtime_epoch": 1688449268, "localtime": "2023-07-04 11:11"}, "current": {"last_updated_epoch": 1688448600, "last_updated": "2023-07-04 11:00", "temp_c": 24.0, "temp_f": 75.2, "is_day": 1, "condition": {"text": "Partly cloudy", "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png", "code": 1003}, "wind_mph": 12.5, "wind_kph": 20.2, "wind_degree": 240, "wind_dir": "WSW", "pressure_mb": 1011.0, "pressure_in": 29.85, "precip_mm": 0.4, "precip_in": 0.02, "humidity": 78, "cloud": 50, "feelslike_c": 25.9, "feelslike_f": 78.5, "vis_km": 6.0, "vis_miles": 3.0, "uv": 5.0, "gust_mph": 23.5, "gust_kph": 37.8}}'

In [83]:
def get_natural_response(content):
    convert_prompt = f'convert this resilts from weather api to a natural english sentensce: {content}'
    messages.append({'role':'user','content':convert_prompt})
    convert_prompt_response = chat_completion_request(messages)
    print(f"Received Message: {convert_prompt_response.json()}")
    new_assistant_message = convert_prompt_response.json()["choices"][0]["message"]
    messages.append(new_assistant_message)
    content = new_assistant_message['content']
    print(f"natural response: \n{content}")
    return content

content=get_natural_response(content)


Received Message: {'id': 'chatcmpl-7YTEsaryp9V2DmLPt26owzaj7Gy6V', 'object': 'chat.completion', 'created': 1688449902, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'The current weather in Bengaluru, Karnataka, India is partly cloudy with a temperature of 24.0°C (75.2°F). The wind is blowing at 20.2 kph (12.5 mph) from the WSW direction. The pressure is at 1011.0 mb (29.85 in) and there is a 0.4 mm (0.02 in) of precipitation. The humidity is at 78% with a cloud coverage of 50%. It feels like 25.9°C (78.5°F) and the visibility is 6.0 km (3.0 miles). The UV index is 5.0 and there is a gust of wind at 37.8 kph (23.5 mph).'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 515, 'completion_tokens': 157, 'total_tokens': 672}}
natural response: 
The current weather in Bengaluru, Karnataka, India is partly cloudy with a temperature of 24.0°C (75.2°F). The wind is blowing at 20.2 kph (12.5 mph) from the WSW direction. The pressure is 